In [1]:
import os
import re 
from glob import iglob
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sb
from sklearn import linear_model
from urllib.request import urlopen
from bs4 import BeautifulSoup

#### Setting up the Base path and making a dummy dataframe

In [2]:
base_path = r'C:/Users/Manan Arora/Desktop/Plaksha TLF/DataX Project/Agriculture/Datatsets/SHC_StateWise/Punjab/**/*.xml'
global_soil_database_df = pd.DataFrame()
soil_data_labels = ['SampleNo', 'SampleCollectionDate', 'LandArea', 'Irrigation', 'Longitude',
                    'Latitude', 'pH', 'EC', 'Organic Carbon (OC)', 'Available Nitrogen (N)', 
                    'Available Phosphorus (P)', 'Available Potassium (K)', 'Available Sulphur (S)',
                    'Available Zinc (Zn)', 'Available Boron (B)', 'Available Iron (Fe)', 
                    'Available Manganese (Mn)', 'Available Copper (Cu)']
soil_database_df = pd.DataFrame(columns=soil_data_labels)

#### Get basic Soil Health Card Infromation from each sample 

In [3]:
for file_path in iglob(base_path, recursive=True):
    #print(file_path)
    tree = ET.parse(file_path)    
    root = tree.getroot()
    display_iter = root.iter('{SoilHealthCard}Details1')
    shc_info = next(display_iter)
    GeoPosition = shc_info.get('Textbox6')
    pattern = '\d+\.\d+'
    location = re.findall(pattern, GeoPosition)
    sample_data = [shc_info.get('Sample_No'), shc_info.get('Sample_Collection_Date'),shc_info.get('Land_Area'),shc_info.get('Irrigation_Rainfed1'),location[0], location[1]]
    display_iter = display_iter.__iter__()
    for temp_test_data in display_iter:
        sample_data.append(temp_test_data.get("TestValue1") + " " + temp_test_data.get('Unit') + ": " +  temp_test_data.get("Rating"))
    soil_database_df = soil_database_df.append(pd.DataFrame([sample_data], columns=soil_data_labels),ignore_index=True)

In [4]:
#Saving it to a CSV
soil_database_df.to_csv(r"C:\Users\Manan Arora\Desktop\Plaksha TLF\DataX Project\Agriculture\Datatsets\SHC_Punjab_Amritsar_Ajnala_Basic.csv",index = False)

#### Getting General Soil Fertliser Recomendations per Sample

In [5]:
sample_nos = []
parameters = []
soils = []
sprays = []

for file_path in iglob(base_path, recursive=True):
    tree = ET.parse(file_path)
    root = tree.getroot()    
    display_iter1 = root.iter('{SoilHealthCard}Details1')
    shc_info = next(display_iter1)
    sample_no = shc_info.get('Sample_No')
    display_iter2 = root.iter('{SoilHealthCard}Details')
    display_iter2 = display_iter2.__iter__()
    #next(display_iter2)
    for temp_test_data in display_iter2:
        sample_nos +=[sample_no]
        parameters += [temp_test_data.get('TestName')]
        soils += [temp_test_data.get('Textbox12')]
        sprays += [temp_test_data.get('Textbox13')]
general_reco = pd.DataFrame(data = {'SampleNo':sample_nos, 'Parameters':parameters,'Fertilizer_Recommendations_ThroughSoil':soils,'Fertilizer_Recommendations_ThroughSpray':sprays})

In [6]:
#Saving it to a CSV
general_reco.to_csv(r"C:\Users\Manan Arora\Desktop\Plaksha TLF\DataX Project\Agriculture\Datatsets\SHC_Punjab_Amritsar_Ajnala_Recomendations.csv",index = False)

#### Getting Crop Specific Fertiliser Recommendations per Sample

In [7]:
sample_nos = []
crop_names = []
fert_comb1 = []
fert_comb2 = []
for file_path in iglob(base_path, recursive=True):
    tree = ET.parse(file_path)
    root = tree.getroot()    
    display_iter1 = root.iter('{SoilHealthCard}Details1')
    shc_info = next(display_iter1)
    sample_no = shc_info.get('Sample_No')
    display_iter2 = root.iter('{SoilHealthCard}Details4')
    display_iter2 = display_iter2.__iter__()
    next(display_iter2)
    for temp_test_data in display_iter2:
        sample_nos +=[sample_no]
        try:
            crop_names += [temp_test_data.get("Crop_Name1")]
            try:
                fert_comb1+= [temp_test_data.get("Comb1_Fert1_name") +  ": " + temp_test_data.get("Comb1_Fert1_Mod_dose") + " | " + temp_test_data.get("Comb1_Fert2_name") + ": " +  temp_test_data.get("Comb1_Fert2_Mod_dose") + " | " + temp_test_data.get("Comb1_Fert3_name") + ": " +  temp_test_data.get("Comb1_Fert3_Mod_dose")]
            except:
                fert_comb1+=["None"]
            try: 
                fert_comb2+= [temp_test_data.get("Comb2_Fert1_name") +  ": " + temp_test_data.get("Comb2_Fert1_Mod_dose") + " | " + temp_test_data.get("Comb2_Fert2_name") + ": " +  temp_test_data.get("Comb2_Fert2_Mod_dose") + " | " + temp_test_data.get("Comb2_Fert3_name") + ": " +  temp_test_data.get("Comb2_Fert3_Mod_dose")]
            except:
                fert_comb2+=["None"]
        except:
            crop_names += [""] 
fert_reco = pd.DataFrame(data = {'SampleNo': sample_nos, 'Crop_Name':crop_names,'Fertiliser_Combination_1': fert_comb1 , 'Fertiliser_Combination_2': fert_comb2})            

In [8]:
#Saving it to a CSV
fert_reco.to_csv(r"C:\Users\Manan Arora\Desktop\Plaksha TLF\DataX Project\Agriculture\Datatsets\SHC_Punjab_Amritsar_Ajnala_Fertilizers.csv",index = False)